#### Folium Map
The following is an interactive map which shows the distance between each substation and outage. The code provides an html file which can later be served on our website. 

In [ ]:
import folium
import sys
import pandas as pd
import folium.plugins as fg

# Allow importing from parent directory by temporarily moving the CWD up one level
# Very hacky, but there literally isn't a simpler way (in Jupyter)
sys.path.append("..")
from common import get_dataframe_from_pipeline

outages = get_dataframe_from_pipeline("../pipeline/3.csv.gz")
# Drop the path back down after import
sys.path.pop()

substations = pd.read_csv("../_datasets/BCSubstationLocations.csv")
populatedPlaces = pd.read_csv("../_datasets/BCPopulatedPlaces.csv")

In [ ]:
# create an open street map of the lower mainland
map = folium.Map(location=(49.1934, -122.89), zoom_start=10)

In [ ]:
# create a layer for populated places, substations, outages, and lines
populatedPlaceLayer = folium.FeatureGroup(name="Populated Places")
substationsLayer = folium.FeatureGroup(name="Substations")
outagesLayer = folium.FeatureGroup(name="Outages")

# TODO change what info in popup it displays?

In [ ]:
# markers for substations
for (index, row) in substations.iterrows():
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = folium.Popup(f"This is the '{row['name']}' substation.", max_width=250),
        tooltip = 'substation',
        icon = folium.Icon(color='blue', icon='tint')
    ).add_to(substationsLayer) # add to the substations layer

In [ ]:
# markers for populated places
for (index, row) in populatedPlaces.iterrows():
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = folium.Popup(f"This populated place is located in {row['geographical name']}.", max_width=250),
        tooltip = 'populated place',
        icon = folium.Icon(color='cadetblue', icon='home')
    ).add_to(populatedPlaceLayer) # add to the populated places layer

In [ ]:

# markers for outages
for (index, row) in outages.iterrows():
    folium.Marker(
        location = [row['outageLatitude'], row['outageLongitude']],
        popup = folium.Popup(f"The outage id '{row['outageId']}' occurred at {row['dateOn']}. It is tagged with the cause: {row['outageCause']}", max_width=250),
        tooltip = 'outage',
        icon = folium.Icon(color='red', icon='flash')
    ).add_to(outagesLayer) # add to the outages layer

    outageCoordinates = [row['outageLatitude'], row['outageLongitude']]
    nearestSubstation = substations.set_index("id").loc[row['nearestSubstationId']]
    substationCoordinates = [nearestSubstation['latitude'], nearestSubstation['longitude']]

    # draw line for substation and outages
    folium.PolyLine(
        locations = [outageCoordinates, substationCoordinates], 
        color = "#6a1ca6", 
        popup = folium.Popup(f"This line connects '{nearestSubstation['name']}' substation and outage id '{row['outageId']}'. It is {round(row['outageToSubstationDistance'], 2)} km long.", max_width=250),
        weight = 3, 
        opacity = 1
    ).add_to(substationsLayer) # add to the sub lines layer

    # draw line for populated places to outages
    nearestPop = populatedPlaces.set_index("id").loc[row['nearestPopulatedPlaceId']]
    popCoordinates = [nearestPop['latitude'], nearestPop['longitude']]
    folium.PolyLine(
        locations = [outageCoordinates, popCoordinates], 
        color = "#a085d6",
        popup = folium.Popup(f"This line connects {nearestPop['geographical name']} and outage id '{row['outageId']}'. It is {round(row['outageToPopulatedPlaceDistance'], 2)} km long.", max_width=250),
        weight = 3, 
        opacity = 1
    ).add_to(populatedPlaceLayer) # add to the pop lines layer

In [ ]:
# add layers to map
populatedPlaceLayer.add_to(map)
substationsLayer.add_to(map)
outagesLayer.add_to(map)

# LayerControl allows toggling between layers
folium.LayerControl().add_to(map)

map


Resources :

[Tutorial (YT)](https://www.youtube.com/watch?v=X8CoVymMOLw)

[Making the markings](https://python-visualization.github.io/folium/latest/getting_started.html)

[Making the markings + maps (YT)](https://www.youtube.com/watch?v=t9Ed5QyO7qY)

[PolyLine](https://python-visualization.github.io/folium/latest/user_guide.html)

[Max width adjustment for pop-up box](https://stackoverflow.com/questions/63152298/updating-folium-changed-the-popup-box-width)

You can find the valid map icons [here](https://getbootstrap.com/docs/3.3/components/).

Note : Icons colour have to be either of these : 'green', 'darkred', 'red', 'lightblue', 'darkgreen', 'white', 'lightgreen', 'orange', 'purple', 'lightred', 'black', 'beige', 'blue', 'cadetblue', 'gray', 'lightgray', 'pink', 'darkpurple', 'darkblue'

In [ ]:
import os
# Save the map
# Make directory if it doesn't exist
if not os.path.exists("html"):
  os.makedirs("html")

map.save('html/mapOfDistance.html')

In [ ]:
# opens the map in your browser
# !open mapOfDistance.html